In [1]:
import pandas as pd
import numpy as np
import re
import datetime as dt
import pytz

In [65]:
class TimePattern:
    def __init__(self,pattern_path=None,tz='America/New_York'):
        self._set_timeZone(tz)
        self._load_mapping(pattern_path)
        
    
    def process(self, sentence):
        current = dt.datetime.now()
        current = self.tz.localize(current)
        times = re.findall(self.re_ext, sentence)
        result = []
        for each in times:
            future = self.evl(self.dict_ext[each])
            gap = (future - current).total_seconds()
            result.append({'pattern':each, 'time':future, 'gapS':gap, 'gapH':gap/3600})
        return result
    
    def evl(self, expression):
        current = dt.datetime.now()
        current = self.tz.localize(current)
        exp_week = re.findall(r'-.+W-.+w',expression)
        exp_ymd = re.findall(r'.+y-.+m.+d',expression)

        history = self._pros_second(expression, current)
        history = self._pros_minute(expression, current, history)
        history,shift = self._pros_hour(expression, current, history)

        if exp_week:
            history = self._pros_weekDay(expression,current, history)
            history = self._pros_week(expression,current, history)
            history = self._pros_year(expression, current,history)
            future = self.create_from_W(history)  
        elif exp_ymd:
            history = self._pros_day(expression, current,history)
            history = self._pros_month(expression, current,history)
            history = self._pros_year(expression, current,history)
            future = self.create_from_D(history)
        if future.tzinfo is None:
            future = self.tz.localize(future)
        if not shift:
            future = future - self.delta
        return future
    
    def _load_mapping(self, pattern_path):
        df = pd.read_csv('mapping.csv')
        # create length
        df['length'] = df.key_word.apply(lambda x: len(x))
        df = df.sort_values(['length','key_word'], ascending=False)
        df_series = pd.Series(index=df.key_word.values, data=df.expression.values)
        df_dict = df_series.to_dict()
        self.serires = df_series
        self.re_ext = r'|'.join(self.serires.index.values)
        self.dict_ext = df_dict
        
    def _set_timeZone(self,tz):
        utc = pytz.utc
        self.tz = pytz.timezone(tz)
        now = dt.datetime.now()
        utc_now = utc.localize(now)
        tz_now = self.tz.localize(now)
        delta =  utc_now - tz_now
        hours = round(delta.total_seconds() / 3600)
        self.delta = dt.timedelta(hours=hours)
        
    def _pros_second(self, expression, current, history={'microsecond':0}):
        history = history.copy()
        S = current.second
        reexp = r'M:.+S'
        extract = re.findall(reexp,expression)[0]
        # M:+1S
        if extract[2:-1] == '?':
            history.update({'second':S})
            return history
        elif extract[2] == '+':
            gap = int(extract[3:-1])
            create = current + dt.timedelta(seconds=gap) 
            create = create.replace(**history)
            return create 
        elif extract[2] == '-':
            gap = int(extract[3:-1])
            create =  current - dt.timedelta(seconds=gap)
            create = create.replace(**history)
            return create
        else:
            second = int(extract[2:-1])
            history.update({'second':second})
            return history
        
    def _pros_minute(self, expression, current, history = {}):
        if isinstance(history,dt.datetime):
            return history
        history = history.copy()
        M = current.minute
        reexp = r'H:.+M'
        extract = re.findall(reexp,expression)[0]
        # H:?M
        if extract[2:-1] == '?':
            history.update({'minute':M})
            return history
        elif extract[2] == '+':
            gap = int(extract[3:-1])
            create = current + dt.timedelta(minutes=gap) 
            create = create.replace(**history)
            return create
        elif extract[2] == '-':
            gap = int(extract[3:-1])
            create = current - dt.timedelta(minutes=gap) 
            create = create.replace(**history)
            return create
        else:
            minute = int(extract[2:-1])
            history.update({'minute':minute})
            return history
        
    def _pros_hour(self, expression, current, history = {}):
        shift = True
        if isinstance(history,dt.datetime):
            return history, shift
        history = history.copy()

        H = current.hour
        reexp = r'[dw]-.+H'
        extract = re.findall(reexp,expression)[0]
    #     d-?H
        if extract[2:-1] == '?':
            history.update({'hour':H})
            return history,shift
        elif extract[2] == '+':
            gap = int(extract[3:-1])
            create = current + dt.timedelta(hours=gap)
            create = create.replace(**history)
            return create,shift
        elif extract[2] == '-':
            gap = int(extract[3:-1])
            create = current - dt.timedelta(hours=gap)
            create = create.replace(**history)
            return create,shift
        else:
            shift = False
            hour = int(extract[2:-1])
            history.update({'hour':hour})
            return history,shift
        
    def _pros_day(self, expression, current, history={}):
        if isinstance(history,dt.datetime):
            return history
        history = history.copy()
        d = current.day
        reexp = r'm-.+d'
        extract = re.findall(reexp,expression)[0]
        if extract[2:-1] == '?':
            history.update({'day':d})
            return history
        elif extract[2] == '+':
            gap = int(extract[3:-1])
            create = current + dt.timedelta(days=gap) 
            create = create.replace(**history)
            return create
        elif extract[2] == '-':
            gap = int(extract[3:-1])
            create = current - dt.timedelta(days=gap) 
            create = create.replace(**history)
            return create
        else:
            day = int(extract[2:-1])
            history.update({'day':day})
            return history
        
    def _pros_month(self, expression, current, history):
        if isinstance(history,dt.datetime):
            return history
        history = history.copy()
        adjust_year = 0
        m = current.month
        reexp = r'y-.+m'
        extract = re.findall(reexp,expression)[0]
        if extract[2:-1] == '?':
            history.update({'month':m})
            return history
        elif extract[2] == '+':
            cur = int(extract[3:-1]) + m
            if cur > 12:
                adjust_year = int(cur / 12)
                cur = cur % 12
                if cur == 0:
                    cur = 12
                    adjust_year -= 1
            history.update({'year':adjust_year})
            history.update({'month':cur})
            return history
        elif extract[2] == '-':
            cur = m - int(extract[3:-1])
            if cur < 1:
                adjust_year = int(cur / 12) - 1
                cur = cur % 12
                if cur == 0:
                    cur = 12
            history.update({'year':adjust_year})
            history.update({'month':cur})
            return history
        else:
            history.update({'month':int(extract[2:-1])})
            return history
        
    def _pros_year(self, expression, current, history):
        if isinstance(history,dt.datetime):
            return history
        history = history.copy()
        adjust_year = history.get('year')
        if adjust_year is None:
            adjust_year = 0
        y = current.year
        reexp = r'.+y-'
        extract = re.findall(reexp,expression)[0]
        if extract[0:-2] == '?':
            history.update({'year':y+adjust_year})
            return history
        elif extract[0] == '+':
            gap = int(extract[1:-2])
            history.update({'year':y+adjust_year+gap})
            return history
        elif extract[0] == '-':
            gap = int(extract[1:-2])
            history.update({'year':y+adjust_year-gap})
            return history
        else:
            history.update({'year':int(extract[:-2])+adjust_year})
            return history
        
    def _pros_weekDay(self, expression, current, history):
        history = history.copy()
        w = current.isocalendar()[2] 
        reexp = r'W-.+w'
        extract = re.findall(reexp,expression)[0]

        # W-+1w
        if extract[2:-1] == '?':
            history.update({'weekday':str(w)})
            return history
        elif extract[2] == '+':
            rep = str(w + int(extract[3:-1]))
            history.update({'weekday':rep})
            return history
        elif extract[2] == '-':
            rep = str(w - int(extract[3:-1]))
            history.update({'weekday':rep})
            return history
        else:
            rep = extract[2:-1]
            history.update({'weekday':rep})
            return history
        
    def _pros_week(self, expression, current, history):
        history = history.copy()
        year_adjust = 0
        W = current.isocalendar()[1] 
        reexp = r'y-.+W'
        extract = re.findall(reexp,expression)[0]

        # y-+1W
        if extract[2:-1] == '?':
            rep = str(W)
            history.update({'week':rep})
            return history
        elif extract[2] == '+':
            cur = W + int(extract[3:-1])
            if cur > 53:
                year_adjust = int(cur / 53)
                cur = cur % 53
            rep = str(cur)
            history.update({'year':year_adjust})
            history.update({'week':rep})
            return history
        elif extract[2] == '-':
            cur = W - int(extract[3:-1])
            if cur < 0:
                year_adjust = int(cur / 53) -1
                cur = cur % 53
            rep = str(cur)
            history.update({'year':year_adjust})
            history.update({'week':rep})
            return history
        else:
            rep = extract[2:-1]
            history.update({'week':rep})
            return history
    
    def create_from_D(self, history):
        if isinstance(history,dt.datetime):
            return history
        return dt.datetime(**history)
    
    def create_from_W(self, history):
        expression ='{}y-{}W-{}w-{}H:{}M:{}S'.format(history['year'],
                                                     history['week'],
                                                     history['weekday'],
                                                     history['hour'],
                                                     history['minute'],
                                                     history['second'])
        eval_time = dt.datetime.strptime(expression, "%Yy-%WW-%ww-%HH:%MM:%SS")
        return eval_time

In [66]:
t = TimePattern(pattern_path='mapping.csv')

In [70]:
%%timeit
a = t.process('我明天下午哦不不后天下午就还')

112 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [72]:
t.process('我明天下午哦不不后天下午就还')

[{'gapH': 23.37369215916667,
  'gapS': 84145.291773,
  'pattern': '明天下午',
  'time': datetime.datetime(2018, 6, 26, 19, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>)},
 {'gapH': 47.373692159166666,
  'gapS': 170545.291773,
  'pattern': '后天下午',
  'time': datetime.datetime(2018, 6, 27, 19, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>)}]

In [64]:
84608.408779 / 3600

23.502335771944445

In [18]:
%%timeit
a = t.evl('?y-+2W-3w-12H:00M:00S')


85.4 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [21]:
%%timeit
a = t.evl('?y-?m-?d-?H:00M:00S')

61.5 µs ± 895 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [23]:
df = pd.DataFrame([{'key_word':'明天','expression':'?y-?m-+1d-12H:00M:00S'}])

In [25]:
df.to_csv('mapping.csv',encoding='utf-8',index=False)

In [51]:
df = pd.read_csv('mapping.csv')
# create length
df['length'] = df.key_word.apply(lambda x: len(x))
df = df.sort_values(['length','key_word'], ascending=False)
df_series = pd.Series(index=df.key_word.values, data=df.expression.values)
df_dict = df_series.to_dict()

In [54]:
df_series.index.values

array(['明天下午', '后天下午', '两周后', '一周后', '2周后', '1周后', '明天', '后天'], dtype=object)

In [49]:
df = df.sort_values(['length','key_word'], ascending=False)

,expression,key_word,length
1,?y-?m-+1d-15H:00M:00S,明天下午,4
2,?y-?m-+2d-15H:00M:00S,后天下午,4
7,?y-+2W-?w-12H:00M:00S,两周后,3
4,?y-+1W-?w-12H:00M:00S,一周后,3
6,?y-+2W-?w-12H:00M:00S,2周后,3
5,?y-+1W-?w-12H:00M:00S,1周后,3
0,?y-?m-+1d-12H:00M:00S,明天,2
3,?y-?m-+2d-12H:00M:00S,后天,2
